In [12]:
import keras
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, roc_auc_score

class_names = ["confirmed", "crossedout", "empty"]

In [13]:
model = keras.saving.load_model("models/omr_model.keras")
test_ds =  tf.data.Dataset.load("datasets/ds_test")

In [14]:
true_labels = []
for _, label in test_ds:
    true_labels.append(label.numpy())

true_labels = np.concatenate(true_labels)

2024-09-11 22:00:06.565615: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
results = model.predict(test_ds,batch_size=256)

420/420 ━━━━━━━━━━━━━━━━━━━━ 15s 30ms/step


In [16]:
predicted_labels = np.argmax(results, axis=1)

In [17]:
_, predicted_ammount = np.unique(predicted_labels, return_counts=True)

In [18]:
precision, recall, fscore, support = precision_recall_fscore_support(true_labels, predicted_labels, average=None)

In [19]:
auc = roc_auc_score(true_labels, results, average=None, multi_class='ovr')

In [20]:
acc = accuracy_score(true_labels, predicted_labels) * 100

print(f"Test dataset accuracy: {acc:.2f}%")

Test dataset accuracy: 99.54%


In [21]:
import pandas as pd

pd.DataFrame(
    {"class": class_names, "support": support, "predicted_ammount": predicted_ammount, "precision": precision, "recall": recall, "fscore": fscore, "auc": auc}
)

,class,support,predicted_ammount,precision,recall,fscore,auc
0,confirmed,4463,4460,0.995067,0.994398,0.994733,0.999764
1,crossedout,80,90,0.666667,0.750000,0.705882,0.986069
2,empty,8876,8869,0.998872,0.998085,0.998478,0.999975


In [22]:
cm = confusion_matrix(true_labels, predicted_labels)

df = pd.DataFrame(cm, index=class_names, columns=class_names)
df.index.name = "True"
df.columns.name = "Predicted"
df

Predicted,confirmed,crossedout,empty
True,,,
confirmed,4438,19,6
crossedout,16,60,4
empty,6,11,8859
